In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install soundfile

In [0]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [0]:
train_path = '/content/gdrive/My Drive/Hack/Train/'
file_path  = '/content/gdrive/My Drive/Hack/train_label.csv'
x = []
y = []

In [0]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
    def extract_feature(file_name, mfcc, chroma, mel):
        with soundfile.SoundFile(file_name) as sound_file:
            X = sound_file.read(dtype="float32")
            sample_rate=sound_file.samplerate
            if chroma:
                stft=np.abs(librosa.stft(X))
            result=np.array([])
            if mfcc:
                mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
                result=np.hstack((result, mfccs))
            if chroma:
                chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
                result=np.hstack((result, chroma))
            if mel:
                mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
                result=np.hstack((result, mel))
        return result

In [0]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2, train_path = '/content/gdrive/My Drive/Hack/Train/', file_path  = '/content/gdrive/My Drive/Hack/train_label.csv'):
    global x
    global y
    ffile = open(file_path, "r")
    lines = ffile.readlines()[1:]
    ffile.close()
    for line in lines:
      lst_split = line.rstrip().split(',') #line.split(',')
      file_name = lst_split[0]
      class_label = lst_split[1]

      file_name = train_path+file_name
      feature=extract_feature(file_name, mfcc=True, chroma=True, mel=True)
      x.append(feature)
      y.append(class_label)
    return train_test_split(np.array(x), y, test_size=test_size)

In [0]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.10,train_path = '/content/gdrive/My Drive/Hack/Train/', file_path  = '/content/gdrive/My Drive/Hack/train_label.csv')

Dump pickle files


In [0]:
with open('/content/gdrive/My Drive/data/features.csv', 'wb+') as file:
  pickle.dump(x, file)

with open('/content/gdrive/My Drive/data/labels.csv', 'wb+') as file:
  pickle.dump(y, file)

Loading pickle file

In [0]:
with open(r"/content/gdrive/My Drive/data/features.csv","rb") as input_file:
   features = pickle.load(input_file)

with open(r"/content/gdrive/My Drive/data/labels.csv","rb") as input_file:
   labels = pickle.load(input_file)

In [0]:
labels

[]

V. Audio data augmentation


In [0]:
#labels = np.asarray(labels).ravel()

In [0]:
# Cast labels to array
labels = np.asarray(labels)

In [0]:
features = np.asarray(features)

In [0]:
features.shape

(0,)

In [0]:
labels.shape


(31374,)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state = 0)


In [0]:
number_of_samples = features.shape[0]
training_samples = int(number_of_samples * 0.8)
validation_samples = int(number_of_samples * 0.1)
test_samples = int(number_of_samples * 0.1)

In [0]:
training_samples

4183

V. Audio data augmentation

In [0]:
# Number of augmented data
nb_augmented = 2

# Function to add noise to a signals with a desired Signal Noise ratio (SNR)
def noisy_signal(signal, snr_low=15, snr_high=30, nb_augmented=2):
    
    # Signal length
    signal_len = len(signal)

    # Generate White noise
    noise = np.random.normal(size=(nb_augmented, signal_len))
    
    # Compute signal and noise power
    s_power = np.sum((signal / (2.0 ** 15)) ** 2) / signal_len
    n_power = np.sum((noise / (2.0 ** 15)) ** 2, axis=1) / signal_len
    
    # Random SNR: Uniform [15, 30]
    snr = np.random.randint(snr_low, snr_high)
    
    # Compute K coeff for each noise
    K = np.sqrt((s_power / n_power) * 10 ** (- snr / 10))
    K = np.ones((signal_len, nb_augmented)) * K
    
    # Generate noisy signal
    return signal + K.T * noise

In [0]:
# Generate noisy signals from signal list
print("Data Augmentation: START")
augmented_signal = list(map(noisy_signal, features))
print("Data Augmentation: END!")

Data Augmentation: START
Data Augmentation: END!


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
...     features, labels, test_size=0.2)

In [0]:
# Lts do data normalization 
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

In [0]:
# Lets few preparation steps to get it into the correct format for Keras 
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [0]:
# Encode Label from categorical to numerical
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(np.ravel(y_train)))
y_test = np_utils.to_categorical(lb.transform(np.ravel(y_test)))

In [0]:

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [0]:
number_of_samples = features.shape[0]
training_samples = int(number_of_samples * 0.7)
validation_samples = int(number_of_samples * 0.1)
test_samples = int(number_of_samples * 0.2)

In [0]:
from keras.utils import np_utils, to_categorical

In [0]:
labels = to_categorical(labels)
labels.shape

(5229, 6)

In [0]:
X_train.shape

(4183, 1, 180)

In [0]:
y_train.shape

(4183, 6)

In [0]:
from keras.layers import Bidirectional, CuDNNLSTM

### Time Distributed ConvNet imports ###
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, TimeDistributed, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from keras.utils import np_utils
from keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout
from keras.layers import BatchNormalization
from keras.layers import *
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.optimizers import adam


In [0]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b
        eij = K.tanh(eij)
        a = K.exp(eij)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [0]:
# Neural network model
#input_shape = X_train
#Input(shape=(32,))
optimizer = adam(0.005, beta_1=0.1, beta_2=0.001, amsgrad=True)
n_classes = 6

model = Sequential()
model.add(Bidirectional(CuDNNLSTM(180, return_sequences=True), input_shape=(180, 1)))
model.add(Attention(180))
model.add(Dropout(0.2))
model.add(Dense(400))
model.add(ELU())
model.add(Dropout(0.2)) 
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])


In [0]:
# Train model
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=10)
hist = model.fit(np.expand_dims(features[:training_samples],-1), 
                labels[:training_samples], 
                validation_data=(np.expand_dims(features[training_samples:training_samples+validation_samples], -1),
                labels[training_samples:training_samples+validation_samples]), 
                batch_size=64,
                epochs=500,
                callbacks = [es])

Train on 4183 samples, validate on 522 samples
Epoch 1/500
4183/4183 [==============================] - 4s 880us/step - loss: 1.6198 - acc: 0.3191 - val_loss: 1.5353 - val_acc: 0.3103
Epoch 2/500
4183/4183 [==============================] - 2s 515us/step - loss: 1.5169 - acc: 0.3526 - val_loss: 1.5373 - val_acc: 0.3487
Epoch 3/500
4183/4183 [==============================] - 2s 516us/step - loss: 1.4930 - acc: 0.3765 - val_loss: 1.5337 - val_acc: 0.3257
Epoch 4/500
4183/4183 [==============================] - 2s 515us/step - loss: 1.4761 - acc: 0.3887 - val_loss: 1.3866 - val_acc: 0.4195
Epoch 5/500
4183/4183 [==============================] - 2s 514us/step - loss: 1.4407 - acc: 0.4071 - val_loss: 1.4213 - val_acc: 0.4215
Epoch 6/500
4183/4183 [==============================] - 2s 515us/step - loss: 1.4501 - acc: 0.4021 - val_loss: 1.3828 - val_acc: 0.4195
Epoch 7/500
4183/4183 [==============================] - 2s 511us/step - loss: 1.4293 - acc: 0.4150 - val_loss: 1.3762 - val_acc: 0